In [38]:
import gym
from gym import spaces
import numpy as np
import math
import matplotl

In [39]:
def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [40]:
space = spaces.MultiBinary(3)
print([space.sample() for _ in range(5)])

[array([1, 0, 1], dtype=int8), array([1, 0, 1], dtype=int8), array([0, 1, 1], dtype=int8), array([1, 0, 0], dtype=int8), array([1, 0, 0], dtype=int8)]


In [41]:
class MigraineEnv(gym.Env):
    def __init__(self, num_triggers: int = 2, horizon: int = 20):
        self.num_triggers = num_triggers
        self.horizon = horizon
        self.timestep = 0
        self.observation_space = spaces.Discrete(2)  # 0: no migraine, 1: migraine
        self.action_space = spaces.MultiBinary(num_triggers)  # binary representation
        self.state = self.observation_space.sample()
        self.history = np.zeros((horizon, num_triggers))  # Store action history

    def reset(self):
        self.state = self.observation_space.sample()
        self.timestep = 0
        self.history = np.zeros((self.horizon, self.num_triggers))
        return self.state

    def step(self, action: np.array):
        self.history[self.timestep] = action
        y_nat = np.random.normal(0, 1)  # Sample from N(0, 1)
        h_matrix = np.random.binomial(1, 0.5, (self.num_triggers, self.timestep + 1))
        y_next = y_nat

        for j in range(self.num_triggers):
            for k in range(self.timestep + 1):
                y_next += h_matrix[j, k] * self.history[self.timestep - k, j]

        self.state = 1 if y_next > 0 else 0  # If y_next is positive, there is a migraine
        self.timestep += 1
        done = self.timestep >= self.horizon
        return self.state, 0, done, {}

    def render(self, mode='human'):
        pass

In [42]:
# Simulation
num_episodes = 1
env = MigraineEnv()
states = []
actions = []

for episode in range(num_episodes):
    state = env.reset()
    for t in range(env.horizon):
        action = env.action_space.sample()  # Randomly sample an action
        next_state, _, done, _ = env.step(action)
        states.append(next_state)
        actions.append(action)
        if done:
            break

# Plotting
plt.plot(range(len(states)), states, label='Wellness (y_t+1)')
plt.plot(range(len(actions)), [a[0] for a in actions], label='Action (u_t) for Trigger 1')
plt.plot(range(len(actions)), [a[1] for a in actions], label='Action (u_t) for Trigger 2')
plt.xlabel('Time Step')
plt.ylabel('Value')
plt.legend()
plt.title('Migraine Simulation')
plt.show()

NameError: name 'plt' is not defined

State : {0, 1} means either no migraine or yes migraine. 

Action: {0, 1} means either you are exposed or not exposed.